In [2]:
from typing import Tuple
import os

import nidaqmx
import nidaqmx.system._collections
import nidaqmx.system._collections.physical_channel_collection
import numpy as np
import numpy.typing


from nidaqmx.constants import AcquisitionType, ProductCategory, LineGrouping
from nidaqmx.constants import LoggingMode, LoggingOperation, READ_ALL_AVAILABLE
import nidaqmx.system

from matplotlib import pyplot as plt 


import ImagingRig1P as irp
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd

In [4]:
genotype = 'ER4d_sytjGCaMP7f_CsChrimsontdT'
date = '23_10_2024'
fly = 'fly2'



### Opto pulses

In [10]:
scan = 'opto_pulse'
opto_powers = {'led450': np.nan,
               'led550': np.nan,
               'pump_compensation': np.nan, # PSI
               'pump_pressure': np.nan # PSI
               }

In [11]:
sample_rate = 10000
duration = 30
n_samples = int(sample_rate*duration)
time = np.arange(0,duration,1./sample_rate)


do_data = {'camera_trigger': np.zeros([n_samples,]),
           'led450_trigger': np.zeros([n_samples,]), # imaging
           'led550_trigger': np.zeros([n_samples,]), # opto
           'pump_trigger': np.zeros([n_samples,]), 
           }

# trigger camera
do_data['camera_trigger'][1:5000]=1

# imaging led
do_data['led450_trigger'][:-1]=1



# stim every 5 seconds
inter_stim_interval = 5
init_delay = 3
stim_dur = .1

start_times = np.arange(init_delay,duration, inter_stim_interval)
end_times = start_times+stim_dur
for s, e in zip(start_times,end_times):
    t_s, t_e = np.argmin(np.abs(time-s)), np.argmin(np.abs(time-e))
    do_data['led550_trigger'][t_s:t_e]=1


# set filename
num = 0
out_dir = f"D:\\{genotype}\\{date}\\{fly}\\{scan}_{num}.tdms"
while os.path.exists(out_dir):
    num +=1 
    out_dir = f"D:\\{genotype}\\{date}\\{fly}\\{scan}_{num}.tdms"
print(out_dir)

D:\ER4d_sytjGCaMP7f_CsChrimsontdT\23_10_2024\fly2\opto_pulse_5.tdms


In [12]:
exp = irp.experiment_builder.Experiment(do_data, out_dir,
                                        sample_rate = sample_rate, duration = duration)
exp.run_tasks()

Dev1/port0/line0
Dev1/port0/line2
Dev1/port0/line3
Dev1/port0/line5
(4, 300000)


### Opto and Pump pairing

In [13]:
scan = 'opto_pump_pulse'

sample_rate = 10000
duration = 70
n_samples = int(sample_rate*duration)
time = np.arange(0,duration,1./sample_rate)


do_data = {'camera_trigger': np.zeros([n_samples,]),
           'led450_trigger': np.zeros([n_samples,]), # imaging
           'led550_trigger': np.zeros([n_samples,]), # opto
           'pump_trigger': np.zeros([n_samples,]), 
           }

# trigger camera
do_data['camera_trigger'][1:5000]=1

# imaging led
do_data['led450_trigger'][:-1]=1



# opto stim
inter_stim_interval = .5
init_delay = 3
stim_dur = .1

start_times = np.arange(init_delay,init_delay+60, inter_stim_interval)
end_times = start_times + stim_dur
for s, e in zip(start_times,end_times):
    t_s, t_e = np.argmin(np.abs(time-s)), np.argmin(np.abs(time-e))
    do_data['led550_trigger'][t_s:t_e]=1

# pump stim
init_delay = 3.3
inter_stim_interval = 2
stim_dur = 1
start_times = np.arange(init_delay,init_delay+60, inter_stim_interval)
end_times = start_times + stim_dur
for s, e in zip(start_times,end_times):
    t_s, t_e = np.argmin(np.abs(time-s)), np.argmin(np.abs(time-e))
    do_data['pump_trigger'][t_s:t_e]=1



num = 0
out_dir = f"D:\\{genotype}\\{date}\\{fly}\\{scan}_{num}.tdms"
while os.path.exists(out_dir):
    num +=1 
    out_dir = f"D:\\{genotype}\\{date}\\{fly}\\{scan}_{num}.tdms"
print(out_dir)

D:\ER4d_sytjGCaMP7f_CsChrimsontdT\23_10_2024\fly2\opto_pump_pulse_0.tdms


In [14]:
exp = irp.experiment_builder.Experiment(do_data, out_dir,
                                        sample_rate = sample_rate, duration = duration)
exp.run_tasks()

Dev1/port0/line0
Dev1/port0/line2
Dev1/port0/line3
Dev1/port0/line5
(4, 700000)


Dev1/port0/line0
Dev1/port0/line2
Dev1/port0/line3
Dev1/port0/line5
(4, 500000)


In [74]:
from nptdms import TdmsFile

tdms_file = TdmsFile.read(out_dir)
# group = tdms_file['group name']
# channel = group['channel name']
# channel_data = channel[:]
# channel_properties = channel.properties

In [37]:
analog_inputs = tdms_file['analog_inputs']


In [58]:
channels = analog_inputs.channels()
channels[:]
print(channels)


[<TdmsChannel with path /'analog_inputs'/'Dev1/ai0'>, <TdmsChannel with path /'analog_inputs'/'Dev1/ai1'>, <TdmsChannel with path /'analog_inputs'/'Dev1/ai6'>]


In [53]:
analog_inputs.channels()

[<TdmsChannel with path /'analog_inputs'/'Dev1/ai0'>,
 <TdmsChannel with path /'analog_inputs'/'Dev1/ai1'>,
 <TdmsChannel with path /'analog_inputs'/'Dev1/ai6'>]

In [48]:
time = channel.time_track()

In [51]:
1/(time[1]-time[0])

10000.0